## Introduction

This notebook demonstrates the use of the expectation maximisation Gaussian mixture models (EMGMM) and KMeans for unsupervised learning. It also includes a number of plots to illustrate the clustering of the datapoints to compare KMeans alone with EMGMM initialized using KMeans.

It is also available on GitHub at: https://github.com/mariamingallonMM/AI-ML-W8-clustering


## How

The following are the key steps we take to implement the KMeans and EMGMM algorithms and produce the plots.


1. Read the dataset: We are given n data points {x1,…,xn}, where each xi∈Rd.


2. Implement KMeans algorithm to learn 5 clusters and run for a number of iterations determined by the user. With K-means we are trying to find K centroids {μ1,…,μK} and the corresponding assignments of each data point {c1,…,cn}, where each ci ∈ {1,…,K} and ci indicates which of the K clusters the observation xi belongs to. The objective function that we seek to minimize can be written: 

    > L=∑ni=1∑Kk=11(ci=k)∥xi−μk∥2

3. Implement EMGMM algorithm to learn 5 clusters and run for a number of iterations determined by the user. The Expectation-Maximisation (EM) algorithm shall learn the parameters of a Gaussian mixture model (GMM), that is learning π, μ and Σ. For this model, we assume a generative process for the data where the 'ith' observation is first assigned to one of K clusters according to the probabilities in vector π. The value of observation xi is then generated from one of K multivariate Gaussian distributions, using the mean (μ) and covariance indexed by ci. This process can be expressed as follows:

    > xi|ci∼Normal(μci,Σci),ci∼Discrete(π)

4. Finally, we implement the EM algorithm to maximize the equation below over all parameters π,μ1,…,μK,Σ1,…,ΣK using the cluster assignments c1,…,cn as the hidden data:

    > p(x1,…,xn|π,μ,Σ)=∏ni=1p(xi|π,μ,Σ)

5. Note that the K-means and EG-GM algorithms shall be written to learn a number of clusters (k) specified by the user (k=5 specified in this assignment). We shall also allow the user to specify the number of iterations for running both algorithms (iterations = 10 specified in this assignmen). 

6. The algorithms can be initialized arbitrarily. However, we are initialising K-means centroids by randomly selecting 5 data points (K-memoids). For the EM-GMM, the mean vectors can be initialized in the same fashion, with π initialized to be the uniform distribution and each Σk to be the identity (I) matrix. Note that when initializing GMM we will have first run K-means and we will use the resulting cluster centers as the means of the Gaussian components.

Finally, note that GMM yields a probability distribution over the cluster assignment for each point; whereas K-means gives a single hard assignment.


In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# builtin modules
import sys
import os
import math
from random import randrange
import functools
import operator
import requests
import psutil

# 3rd party modules
import numpy as np
import pandas as pd
import scipy as sp
from scipy.cluster.vq import kmeans2
from scipy.stats import multivariate_normal
from scipy.spatial.distance import cdist
from scipy.special import logsumexp
from scipy import stats

# import plotting modules; not needed in Vocareum
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

from IPython.display import HTML

/kaggle/input/clusteringgmmtest/LICENSE
/kaggle/input/clusteringgmmtest/README.md
/kaggle/input/clusteringgmmtest/hw3_clustering.py
/kaggle/input/clusteringgmmtest/.vs/PythonSettings.json
/kaggle/input/clusteringgmmtest/.vs/ProjectSettings.json
/kaggle/input/clusteringgmmtest/.vs/slnx.sqlite
/kaggle/input/clusteringgmmtest/.vs/VSWorkspaceState.json
/kaggle/input/clusteringgmmtest/outputs/sigma-2-8.csv
/kaggle/input/clusteringgmmtest/outputs/sigma-2-2 (2).csv
/kaggle/input/clusteringgmmtest/outputs/mu-1 (3).csv
/kaggle/input/clusteringgmmtest/outputs/sigma-3-1.csv
/kaggle/input/clusteringgmmtest/outputs/sigma-4-5 (3).csv
/kaggle/input/clusteringgmmtest/outputs/sigma-5-3.csv
/kaggle/input/clusteringgmmtest/outputs/sigma-5-7.csv
/kaggle/input/clusteringgmmtest/outputs/centroids-5.csv
/kaggle/input/clusteringgmmtest/outputs/pi-2 (3).csv
/kaggle/input/clusteringgmmtest/outputs/centroids-9.csv
/kaggle/input/clusteringgmmtest/outputs/sigma-1-6 (2).csv
/kaggle/input/clusteringgmmtest/outputs/s

### Step 1: Import the dataset

In [7]:
data = pd.read_csv("/kaggle/input/clusteringgmmtest/datasets/Clustering_gmm.csv")
data.head()

,Weight,Height
0,67.062924,176.086355
1,68.804094,178.388669
2,60.930863,170.284496
3,59.733843,168.691992
4,65.431230,173.763679


### Define KMeans function

In [8]:
def KMeans(data, k:int = 5, n:int = 10, **kwargs):
    """
    It uses the scipy.klearn2 algorithm to classify a set of observations into k clusters using the k-means algorithm (scipy.kmeans2). It attempts to minimize the Euclidean distance between observations and centroids. Note the following methods for initialization are available:
        - ‘random’: generate k centroids from a Gaussian with mean and variance estimated from the data.
        - ‘points’: choose k observations (rows) at random from data for the initial centroids.
        - ‘++’: choose k observations accordingly to the kmeans++ method (careful seeding)
        - ‘matrix’: interpret the k parameter as a k by M (or length k array for 1-D data) array of initial centroids.
    It is recommended the kmeans algorithm is initialized by randomly selecting 5 data points (minit = "points", with k = 5). Also, note we use the identity matrix for each cluster's covariance matrix for the initialization. We also try initializing data points without replacement: for example, using np.random.choice and set replace = False.
    ------------
    Parameters:
    - data: ndarray of the set of observations to classify, the n data points {x1,…,xn}, where each xi ∈ Rd (shape: n, d; where n: number of rows, d: number of features/columns)
    - k: number of clusters to consider, 5 clusters by default
    - n: number of iterations to consider, 10 iterations by default
    ------------
    Returns:
    - centroids, which are the means for each cluster {μ1,…,μK}
    - labels, are the corresponding assignments of each data point {c1,…,cn}, where each ci ∈ {1,…,K} and ci indicates which of the K clusters the observation xi belongs to.
    - writes the centroids of the clusters associated with each iteration to a csv file, one file per iteration; pass on a path if different from the default being "current working directory" + "outputs"
    """

    data = data.to_numpy() # Convert dataframe to np array
    centroids_list = []
    labels_list = []

    for i in range(n):
        centroids, label = kmeans2(data, k, iter = i+1, minit='points')
        centroids_list.append(centroids)
        labels_list.append(label)
        filename = "centroids-" + str(i+1) + ".csv" #"i" would be each iteration
        if 'path' in kwargs:
            path = kwargs['path']
            filepath = os.path.join(path, filename)
            np.savetxt(filepath, centroids, delimiter=",")
        else:
            path = os.path.join(os.getcwd(), "outputs")
            filepath = os.path.join(path, filename)
            np.savetxt(filepath, centroids, delimiter=",")
    
    return centroids_list, labels_list

### Define EMGMM main functions

Calculate means and covariance of different clusters from k-means prediction

In [9]:
def calculate_mean_covariance(data, centroids, labels):
    """
    Calculates means and covariance of different clusters from k-means prediction. This helps us calculate values for our initial parameters. It takes in our data as well as our predictions from k-means and calculates the weights, means and covariance matrices of each cluster. Note that we shall use the results from each of the iterations of k-means algorithm.
    Note that 'counter' is equivalent to 'cluster_label' provided the clusters are of int type and ranging from '0' to 'k'. We could have simplified counter = cluster_label but we chose not to do so to allow for cases in which the lable is not an integer.
    ------------
    Parameters:
    - data: ndarray of the set of observations to classify, the n data points {x1,…,xn}, where each xi ∈ Rd (shape: n, d; where n: number of rows, d: number of features/columns)
    - centroids: centroids from KMeans method
    - labels: cluster labels from KMeans method, note it includes all the iterations
    - k: number of clusters (also number of Gaussians)
    -------------
    Returns:
    A tuple containing:
        
    - initial_pi: initial array of pik (πk = nk / n) values for each cluster k to input in the E-step of EM algorithm (shape: k,)
    - initial_mean: initial array of mu (μk, mean) values for each cluster kto input in the E-step of EM algorithm (shape: k, d)
    - initial_sigma: initial array of covariance (Σk, sigma) values for each cluster k to input in the E-step of EM algorithm (shape: k, d*d)
        
    """
    # initialize the output ndarrays with zeros, for filling up in loops below
    d = data.shape[1]
    k = len(centroids[0]) # to take number of clusters directly from KMeans
    initial_pi = np.zeros(k)
    initial_mean = np.zeros((k, d))
    initial_sigma = np.zeros((k, d, d))
    
    # get the number of iterations from the lenght of centroids (or labels)
    iterations = len(centroids)
        
    for i in range(iterations):
        iter_centroid = centroids[i]
        iter_labels = labels[i]
        # initialize counter to organize outputs per cluster k (counter = cluster_labels if the latter are int type ranging from 0 to k)
        counter=0
        for cluster_label in np.unique(iter_labels):
            # returns indices of rows estimated to belong to the cluster
            ids = np.where(iter_labels == cluster_label)[0] 

            # calculate pi (π = nk / n) for cluster k (πk)
            nk = data.iloc[ids].shape[0] # number of data points in current gaussian/cluster
            n = data.shape[0] # total number of points/rows in dataset
            initial_pi[counter] = nk / n

            # calculate mean (mu) of points estimated to be in cluster k (μk)
            initial_mean[counter,:] = np.mean(data.iloc[ids], axis = 0)
            de_meaned = data.iloc[ids] - initial_mean[counter,:]
            
            # calculate covariance (Σ, sigma) of points estimated to be in cluster k (Σk) 
            initial_sigma[counter, :, :] = np.dot(initial_pi[counter] * de_meaned.T, de_meaned) / nk
            
            counter+=1
        
        #assert np.sum(initial_pi) == 1    
            
    return (initial_pi, initial_mean, initial_sigma)

Call KMeans to obtain the starting centroids and label values and use them as starting parameters for the EMGMM algorithm. 

In [29]:
def initialise_parameters(data, **kwargs):
    """
    Calls the function KMeans to obtain the starting centroids and label values and use them as starting parameters for the EMGMM algorithm. 
    
    ------------    
    Parameters:
    
    - data: ndarray of the set of observations to classify, the n data points {x1,…,xn}, where each xi ∈ Rd (shape: n, d; where n: number of rows, d: number of features/columns)
    
    ----------
    Returns:
    
    A tuple containing:
        
    - initial_pi: initial array of pik (πk = nk / n) values for each cluster k to input in the E-step of EM algorithm (shape: k,)
    - initial_mean: initial array of mu (μk, mean) values for each cluster k to input in the E-step of EM algorithm (shape: k, d)
    - initial_sigma: initial covariance matrices (Σk, sigma), one matrix for each cluster k to input in the E-step of EM algorithm (shape: k, d*d)
        
    """
    
    
    if 'path' in kwargs:
        path = kwargs['path']
    else:
        path = os.path.join(os.getcwd(), "outputs")
    
    centroids, labels =  KMeans(data, path=path)

    (initial_pi, initial_mean, initial_sigma) = calculate_mean_covariance(data, centroids, labels)
        
    return (initial_pi, initial_mean, initial_sigma)




Perform E-step on GMM model

In [11]:
def e_step(data, pi, mu, sigma):
    """
    Performs E-step on GMM model
    
    ------------
    Parameters:
    
    - data: data points in numpy array (shape: n, d; where n: number of rows, d: number of features/columns)
    - pi: weights of mixture components pik (πk = nk / n) values for each cluster k in an array (shape: k,)
    - mu: mixture component mean (μk, mean) values for each cluster k in an array (shape: k, d)
    - sigma: mixture component covariance matrices (Σk, sigma), one matrix for each cluster k (shape: k, d, d)
    
    ----------
    Returns:
    - gamma: probabilities of clusters for datapoints (shape: n, k)
    """

    
    
    n = data.shape[0]
    k = len(pi)
    gamma = np.zeros((n, k))

    # convert np.nan to float(0.0)
    pi = np.nan_to_num(pi)
    mu = np.nan_to_num(mu)
    sigma = np.nan_to_num(sigma)

    x = data.to_numpy() # Convert dataframe to np array

    for cluster in range(k):
        # Posterior Distribution using Bayes Rule
        gamma[ : , cluster] = pi[cluster] * multivariate_normal(mean=mu[cluster,:], cov=sigma[cluster], allow_singular=True).pdf(x)

    gamma = np.nan_to_num(gamma) # convert np.nan to float(0.0)
    # normalize across columns to make a valid probability
    gamma_norm = np.sum(gamma, axis=1)[ : , np.newaxis]
    gamma_norm = np.nan_to_num(gamma_norm)
    gamma /= gamma_norm

    return np.nan_to_num(gamma)


Perform M-step of the GMM to update the priors, pi, means and covariance matrix.

In [12]:
def m_step(data, gamma, sigma):
    """
    Performs M-step of the GMM. It updates the priors, pi, means and covariance matrix.
    -----------
    Parameters:
    
    - data: ndarray of the set of observations to classify, the n data points {x1,…,xn}, where each xi ∈ Rd (shape: n, d; where n: number of rows, d: number of features/columns)
    - gamma: probabilities of clusters for datapoints (shape: n, k)
    - sigma: mixture component covariance matrices (Σk, sigma), one matrix for each cluster k (shape: k, d, d)
    
    ---------
    Returns:
    - pi: updated weights of mixture components pik (πk = nk / n) values for each cluster k in an array (shape: k,)
    - mu: updated mixture component mean (μk, mean) values for each cluster k in an array (shape: k, d)
    - sigma: updated mixture component covariance matrices (Σk, sigma), one matrix for each cluster k (shape: k, d, d)
    """
    n = data.shape[0] # number of datapoints (rows in the dataset), equivalent of gamma.shape[0]
    k = gamma.shape[1] # number of clusters
    d = data.shape[1] # number of features (columns in the dataset)
    
    # convert np.nan to float(0.0)
    gamma = np.nan_to_num(gamma)
    sigma = np.nan_to_num(sigma)

    # calculate pi and mu for each Gaussian
    pi = np.mean(gamma, axis = 0)
    mu = np.dot(gamma.T, data) / np.sum(gamma, axis = 0)[:,np.newaxis]

    x = data.to_numpy() # Convert dataframe to np array

    # update sigma for each Gaussian
    for cluster in range(k):
        x = x - mu[cluster, :] # (shape: n, d)
        gamma_diag = np.diag(gamma[: , cluster])
        x_mu = np.matrix(x)
        gamma_diag = np.matrix(gamma_diag)

        sigma_cluster = x.T * gamma_diag * x
        gamma = np.nan_to_num(gamma) # convert np.nan to float(0.0)
        sigma[cluster,:,:] = (sigma_cluster) / np.sum(gamma, axis = 0)[:,np.newaxis][cluster]

    return pi, mu, sigma

Compute the lower bound loss function for a given dataset, and values of pi, mu and sigma. 


In [13]:
    
def compute_loss_function(data, pi, mu, sigma, tol):
    """
    Computes the lower bound loss function for a given dataset, and values of pi, mu and sigma. 
    -----------  
    Parameters:
    
    - data: ndarray of the set of observations to classify, the n data points {x1,…,xn}, where each xi ∈ Rd (shape: n, d; where n: number of rows, d: number of features/columns)
    - pi: weights of mixture components pik (πk = nk / n) values for each cluster k in an array (shape: k,)
    - mu: mixture component mean (μk, mean) values for each cluster k in an array (shape: k, d)
    - sigma: mixture component covariance matrices (Σk, sigma), one matrix for each cluster k (shape: k, d, d)
    - tol: tolerance for loss to be considered equivalent to zero and hence end iterative process (default 1e-6)
    -----------
    Returns:
    - loss_output: the value of lower bound loss function, as a float.
    
    """
    
    n = data.shape[0] # number of datapoints (rows in the dataset), equivalent of gamma.shape[0]
    k = len(pi) # number of clusters, equivalent to gamma.shape[1]
    loss = np.zeros((n, k)) # set up the array to store the loss values for each iteration

    x = data.to_numpy() # Convert dataframe to np array

    # convert np.nan to float(0.0)
    mu = np.nan_to_num(mu)
    pi = np.nan_to_num(pi)
    sigma = np.nan_to_num(sigma)

    for cluster in range(k):
        dist = multivariate_normal(mu[cluster], sigma[cluster], allow_singular=False)
        loss[:,cluster] = gamma[:,cluster] * (np.log(pi[cluster] + tol) + dist.logpdf(x) - np.log(gamma[:,cluster] + tol))
    
    loss = np.nan_to_num(loss) # convert np.nan to float(0.0)
    loss_output = np.sum(loss)

    return loss_output

Performs the Expectation-Maximisation (EM) algorithm to learn the parameters of a Gaussian mixture model (GMM), that is learning **π**, **μ** and **Σ**. Note that a Gaussian mixture model (GMM) attempts to discover a mixture of multi-dimensional Gaussian probability distributions that best model any input dataset.

In [31]:
def EMGMM(data, k:int = 5, iterations:int = 10, tol:float = 1e-6, **kwargs):
    """
    Performs the Expectation-Maximisation (EM) algorithm to learn the parameters of a Gaussian mixture model (GMM), that is learning **π**, **μ** and **Σ**. A Gaussian mixture model (GMM) attempts to discover a mixture of multi-dimensional Gaussian probability distributions that best model any input dataset. In the simple case, GMMs can be used for finding clusters in the same manner as k-means. For this model, we assume a generative process for the data as follows:
    xi|ci∼Normal(μci,Σci),ci∼Discrete(π)
    
    where:
    - the ith observation is first assigned to one of K clusters according to the probabilities in vector  π, and 
    - the value of observation xi is then generated from one of K multivariate Gaussian distributions, using the mean (μ) and covariance indexed by ci. 
    
    Finally, we implement the EM algorithm to maximize the equation below over all parameters (π,μ1,…,μK,Σ1,…,ΣK) using the cluster assignments (c1,…,cn) as the hidden data:
    
    p(x1,…,xn|π,μ,Σ)=∏ni=1p(xi|π,μ,Σ)
    ------------
    Parameters:
    - data: ndarray of the set of observations to classify, the n data points {x1,…,xn}, where each xi ∈ Rd (shape: n, d; where n: number of rows, d: number of features/columns)
    - k: number of clusters to consider, 5 clusters by default
    - n: number of iterations to consider, 10 iterations by default
    ------------
    Returns:
    The following csv files:
    - pi-[iteration].csv: This is a comma separated file containing the cluster probabilities of the EM-GMM model. The  k th row should contain the  k th probability,  πk , and there should be 5 rows. There should be 10 total files. For example, "pi-3.csv" will contain the cluster probabilities after the 3rd iteration.
    - mu-[iteration].csv: This is a comma separated file containing the means of each Gaussian of the EM-GMM model. The  k th row should contain the  k th mean , and there should be 5 rows. There should be 10 total files. For example, "mu-3.csv" will contain the means of each Gaussian after the 3rd iteration.
    - sigma-[cluster]-[iteration].csv: This is a comma separated file containing the covariance matrix of one Gaussian of the EM-GMM model. If the data is  d -dimensional, there should be  d  rows with  d  entries in each row. There should be 50 total files. For example, "sigma-2-3.csv" will contain the covariance matrix of the 2nd Gaussian after the 3rd iteration.
    """
    
    if 'path' in kwargs:
        path = kwargs['path']
    else:
        path = os.path.join(os.getcwd(), "outputs")
    
    d = data.shape[1] # number of features (columns in the dataset)
    n = data.shape[0] # number of datapoints (rows in the dataset)

    x = data.to_numpy() # Convert dataframe to np array

    # initialise outputs arrays as empty
    centroids_list = []
    predicted_labels_list = []
    
    pi, mu, sigma =  initialise_parameters(data, path=path)

    for i in range(iterations):  
        gamma  = e_step(data, pi, mu, sigma)
        pi, mu, sigma = m_step(data, gamma, sigma)

        filename = "pi-" + str(i + 1) + ".csv"
        filepath = os.path.join(path, filename)
        np.savetxt(filepath, pi, delimiter=",") #this must be done at every iteration
        filename = "mu-" + str(i + 1) + ".csv"
        filepath = os.path.join(path, filename)
        np.savetxt(filepath, mu, delimiter=",")  #this must be done at every iteration
        for cluster in range(k): #k is the number of clusters
            filename = "sigma-" + str(cluster + 1) + "-" + str(i + 1) + ".csv" #this must be done k times for each iteration
            filepath = os.path.join(path, filename)
            np.savetxt(filepath, sigma[cluster], delimiter=",")

        predicted_labels = predict(x, pi, mu, sigma, k)

        # convert np.nan to float(0.0)
        pi = np.nan_to_num(pi)
        mu = np.nan_to_num(mu)
        sigma = np.nan_to_num(sigma)

        # compute centers as point of highest density of distribution
        centroids = np.zeros((k,d))

        for cluster in range(k):
            density = multivariate_normal(mean=mu[cluster], cov=sigma[cluster], allow_singular=True).logpdf(x)
            centroids[cluster, :] = x[np.argmax(density)]

        centroids_list.append(centroids)
        predicted_labels_list.append(predicted_labels)
   
    return centroids_list, predicted_labels_list



Prediction functions to visualise performance of EMGMM algorithm

In [15]:
def predict(data, pi, mu, sigma, k:int = 5):
    """
    Returns predicted labels using Bayes Rule to calculate the posterior distribution
    
    -------------
    Parameters:
    
    - data: ndarray of the set of observations to classify, the n data points {x1,…,xn}, where each xi ∈ Rd (shape: n, d; where n: number of rows, d: number of features/columns)
    - k: number of clusters to consider, 5 clusters by default
    ----------
    Returns:
    
    - labels: the predicted label/cluster based on highest probability gamma.
        
    """
    n = data.shape[0] # number of datapoints (rows in the dataset)
    labels = np.zeros((n, k))

    # convert np.nan to float(0.0)
    pi = np.nan_to_num(pi)
    mu = np.nan_to_num(mu)
    sigma = np.nan_to_num(sigma)
    
    #x = data.to_numpy() # Convert dataframe to np array

    for cluster in range(k):
        labels [:,cluster] = pi[cluster] * multivariate_normal(mean=mu[cluster,:], cov=sigma[cluster], allow_singular=True).pdf(data)
    labels  = labels .argmax(1)

    return labels 
    
def predict_proba(data, pi, mu, sigma, k:int = 5):
    """
    Returns the posterior probability of the predicted label/cluster for each datapoint.
    -------------
    Parameters:
    
    - data: ndarray of the set of observations to classify, the n data points {x1,…,xn}, where each xi ∈ Rd (shape: n, d; where n: number of rows, d: number of features/columns)
    - k: number of clusters to consider, 5 clusters by default
    
    ----------
    Returns:
    
    - post_proba: the posterior probability of the predicted label/cluster for each datapoint.
        
    """
    n = data.shape[0] # number of datapoints (rows in the dataset)
    post_proba = np.zeros((n, k))

    # convert np.nan to float(0.0)
    pi = np.nan_to_num(pi)
    mu = np.nan_to_num(mu)
    sigma = np.nan_to_num(sigma)
    
    #x = data.to_numpy() # Convert dataframe to np array
        
    for cluster in range(k):
        # Posterior Distribution using Bayes Rule, try and vectorise
        post_proba[:,cluster] = pi[cluster] * multivariate_normal(mean=mu[cluster,:], cov=sigma[cluster], allow_singular=True).pdf(data)
    
    return post_proba

### Plotting functions

In [16]:
def plot_simple(dataframe, filetitle:str = 'sampledata'):

    """
    Plots the sample data.
    ------------
    Parameters:
    ------------
    Returns:
    """

    df = dataframe

    names_axis = list(dataframe.columns)

    fig = go.Figure(data=go.Scatter(x=df[names_axis[0]], 
                                        y=df[names_axis[1]],
                                        mode='markers',
                                        marker=dict(
                                        line_width=1,
                                        size = 16),
                                        showlegend=False))  # turn off legend only for this item

    # Give the figure a title
    fig.update_layout(title='Kmeans | Clustering | Project 3',
                        xaxis_title=names_axis[0],
                        yaxis_title=names_axis[1])

    # Show the figure, by default will open a browser window
    fig.show()

def plot_clusters(dataframe, centroids, labels, **kwargs):
    """
    Plots the clusters identified in the data which vary per iteration. It also plots the centroids of the clusters considered by the kmeans algorithm for each iteration. The plot is presented on an interactive browser window which allows for each of the iterations to be turn on/off as desire by the user. 
    
    ------------
    Parameters:
    
    - dataframe: the dataframe of the points to plot (the original dataset)
    - centroids: x,y coordinates of the centroids of each cluster as obtained from KMeans algorithm
    - labels: labels of the cluster each centroid belongs to as obtained from KMeans algorithm
    - names_axis: names of the columns in the dataframe for naming the axis
    ------------
    Returns:
    - a seris of plots illustrating the sample data clustered (k number of clusters) and how the clustering varies per iteration. 
    """

    add_traces =[]

    df = dataframe

    names_axis = list(dataframe.columns)
   
    for i in range(len(centroids)):
        sample_data = go.Scatter(x=df[names_axis[0]], 
                                        y=df[names_axis[1]],
                                        name = 'iteration ' + str(i+1),
                                        mode='markers',
                                        marker=dict(
                                            color=labels[i],
                                            colorscale='Portland',
                                            line_width=1,
                                            size = 12),
                                        text=['cluster ' + str(k+1) for k in labels[i]], # hover text goes here
                                        legendgroup = 'iteration' + str(i+1),
                                        showlegend=True)  # turn off legend only for this item
        add_traces.append(sample_data)

        trace = go.Scatter(x=np.transpose(centroids[i])[0],
                            y=np.transpose(centroids[i])[1],
                            name='centroids iteration ' + str(i+1),
                            mode='markers',
                            marker=dict(
                                symbol='x',
                                #color = k + 1,
                                #colorscale='Greys',
                                line_width=1,
                                size = 20,
                                ),
                            text=['centroid k=' + str(k + 1) for k in range(len(np.transpose(centroids[i])[0]))], # hover text goes here
                            legendgroup = 'centroid iteration' + str(i+1),
                            showlegend=True)  # turn off legend only for this item
        add_traces.append(trace)
        data = go.Data(add_traces)
        fig = go.Figure(data=data)

        # TODO: add circle/ellipses to represent extent of each KMeans / Gaussian
        #w_factor = 0.2 / model.pi.max()
        #for pos, covar, w in zip(model.mu, model.sigma, model.pi):
        #    draw_ellipse(pos, covar, alpha = w)

        # Give the figure a title
        if 'fig_type' in kwargs:
            fig_type = kwargs['fig_type']
            if fig_type == 'kmeans':
                fig_title='Kmeans | Clustering | Project 3 | k = ' + str(len(centroids[i])) + ' clusters, ' + 'total iterations i = ' + str(i + 1)
            if fig_type == 'gmm':
                fig_title='EM Gaussian Mixture Model (initialised from KMeans) | Clustering | Project 3 | k = ' + str(len(centroids[i])) + ' clusters, ' + 'total iterations i = ' + str(i + 1)
            fig.update_layout(title=fig_title,
                                xaxis_title=names_axis[0],
                                yaxis_title=names_axis[1])
        else:
            fig.update_layout(title='Clustering | Project 3 | k = ' + str(len(centroids[i])) + ' clusters, ' + 'total iterations i = ' + str(i + 1),
                                xaxis_title=names_axis[0],
                                yaxis_title=names_axis[1])

    # Show the figure, by default will open a browser window
    fig.show()

    return


### Helper functions

In [17]:
def get_data(filename, **kwargs):
    """
    Read data from a file given its name. Option to provide the path to the file if different from: [./datasets/in].
    ------------
    Parameters:
    - filename: name of the file to be read, to get the data from.
    - kwargs (optional): 
        - 'headers': list of str for the headers to include in the outputs file created
        - 'path': str of the path to where the file is read, specified if different from default ([./datasets/in])
    ------------
    Returns:
    - df: a dataframe of the data
    """
    
    # Define input filepath
    if 'path' in kwargs:
        filepath = kwargs['path']
    else:
        filepath = os.path.join(os.getcwd(),'datasets','out')

    input_path = os.path.join(filepath, filename)

    # If provided, use the title of the columns in the input dataset as headers of the dataframe
    if 'headers' in kwargs:
        # Read input data
        df = pd.read_csv(input_path, names = kwargs['headers'])
    else:
        # Read input data
        df = pd.read_csv(input_path)
       
    return df

### Execute the KMeans and EMGMM functions on the dataset

Plot dataset sample prior to clustering


In [18]:
plot_simple(data)

Run KMeans to get clusters in the data and a csv of clusters per iteration

In [21]:
k = 5
n = iterations = 10
centroids, labels = KMeans(data, k, n, path='./')

Plot results after KMeans

In [22]:
plot_clusters(data, centroids, labels, fig_type='kmeans')

/opt/conda/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:40: DeprecationWarning:

plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.




Run EMGMM to output the required csv files plus the predicted_labels

In [32]:
centroids_emgmm, labels_emgmm = EMGMM(data, k, 10, path='./')

Plot results after EMGMM

In [33]:
plot_clusters(data, centroids_emgmm, labels_emgmm, fig_type='gmm')

# Notes on data repositories

The following datasets have been selected from the UCI Machine Learning Repository for use and testing of the code written for this assignment:

- [3D Road Network (North Jutland, Denmark) Data Set](http://archive.ics.uci.edu/ml/datasets/3D+Road+Network+%28North+Jutland%2C+Denmark%29) This dataset was constructed by adding elevation information to a 2D road network in North Jutland, Denmark (covering a region of 185 x 135 km^2). Elevation values where extracted from a publicly available massive Laser Scan Point Cloud for Denmark (available at : [Web Link] (Bottom-most dataset)). This 3D road network was eventually used for benchmarking various fuel and CO2 estimation algorithms. This dataset can be used by any applications that require to know very accurate elevation information of a road network to perform more accurate routing for eco-routing, cyclist routes etc. For the data mining and machine learning community, this dataset can be used as 'ground-truth' validation in spatial mining techniques and satellite image processing. It has no class labels, but can be used in unsupervised learning and regression to guess some missing elevation information for some points on the road. The work was supported by the Reduction project that is funded by the European Comission as FP7-ICT-2011-7 STREP project number 288254. Refer to Citations & References.


# Citations & References

- [3D Road Network (North Jutland, Denmark) Data Set](http://archive.ics.uci.edu/ml/datasets/3D+Road+Network+%28North+Jutland%2C+Denmark%29) Building Accurate 3D Spatial Networks to Enable Next Generation Intelligent Transportation Systems (Accepted and to be published in June) Proceedings of International Conference on Mobile Data Management (IEEE MDM), June 3-6 2013, Milan, Italy.
- [Build Better and Accurate Clusters with Gaussian Mixture Models](https://www.analyticsvidhya.com/blog/2019/10/gaussian-mixture-models-clustering/) by [Aishwarya Singh](https://www.analyticsvidhya.com/blog/author/aishwaryasingh/)
- [In Depth: Gaussian Mixture Models](https://jakevdp.github.io/PythonDataScienceHandbook/05.12-gaussian-mixtures.html).
- [Clustering with Gaussian Mixture Models](https://pythonmachinelearning.pro/clustering-with-gaussian-mixture-models/)
- [K-Means Clustering in Python: A Practical Guide](https://realpython.com/k-means-clustering-python/)
- [Build Better and Accurate Clusters with Gaussian Mixture Models](https://medium.com/analytics-vidhya/build-better-and-accurate-clusters-with-gaussian-mixture-models-ba9851154b1c)
- [Implement Expectation-Maximization Algorithm(EM) in Python from Scratch](https://towardsdatascience.com/implement-expectation-maximization-em-algorithm-in-python-from-scratch-f1278d1b9137)
- [Gaussian Mixture Modelling (GMM)](https://towardsdatascience.com/gaussian-mixture-modelling-gmm-833c88587c7f)

